In [1]:
import torch

# Local imports
from train import train_model

In [2]:
import sys
print(sys.executable)
print(sys.version)

/opt/miniconda3/bin/python
3.12.2 | packaged by conda-forge | (main, Feb 16 2024, 20:50:58) [GCC 12.3.0]


In [3]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

Using device: cuda:0


In [4]:
# In your train.ipynb cell where `args` is defined:

args = {
    "dataset": "DAVIS",  # Or "KIBA", "CD4C"
    "data_path": "../data",
    "protein_graph_dir": "../data/protein_graphs",  # <--- ADD THIS LINE (or update existing)
    "model_path": '../models/gat_cd4c_davis_graphs.pth', # Example model path
    "seed": 42,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "batch_size": 128, # 128
    "max_epochs": 500, # 500
    "lr": 5e-5,      # ছিল 1e-4
    "weight_decay": 1e-5, # ছিল 2e-4
    "stoppage_epochs": 32, # ছিল 32
    "scheduler_patience": 10,
    "scheduler_factor": 0.5,
    "huber_beta": 0.5,
    "clip_grad_norm": 1.0,
    "use_small_dataset": False, # True for quick testing
    "frac_train": 0.8,
    "frac_validation": 0.1,
    "frac_test": 0.1,
    "max_nodes": 72, # Max nodes for padding
    "num_workers": 4, # Or based on your CPU cores
    
    # Model architecture (these should match defaults in get_parser or be set explicitly)
    "hidden_size": 256,
    "emb_size": 192, 
    "num_layers": 8,
    "num_attn_heads": 12,
    "dropout": 0.1, # ছিল 0.1
    "mlp_dropout": 0.2, # ছিল 0.2
    "pooling_dim": 192,
    "mlp_hidden": 256,
    "use_cross": True, # Or False
    # "include_3d_drug": False, # This is handled by getattr in train.py if not present
}

# Then this line converts it to a namespace
# training_args = argparse.Namespace(**args) 
# train_model(training_args, device)

In [5]:
import argparse

training_args = argparse.Namespace(**args)
train_model(training_args, device)

Found local copy...
Loading...
Done!


Model parameters: 1,302,757
DEBUG load_data: Received dataset_name = 'DAVIS' (type: <class 'str'>)
Attempting to load main interaction data from: ../data/DAVIS_dataset.csv
Cache file ../data/DAVIS_dataset.csv not found. Attempting to load DAVIS via TDC...
INFO: Applying DAVIS-specific Kd transformation to labels.
Loaded and cached DAVIS from TDC to ../data/DAVIS_dataset.csv
Dataset split: Train 20617, Validation 2577, Test 2578


Processing drugs: 100%|██████████| 68/68 [00:00<00:00, 633.85it/s]


INFO: Validating protein graph paths...
INFO: Checking 379 unique Target_IDs from the DataFrame.
  Processed ID 1/379: original='PLK4', sanitized='PLK4' -> FOUND (using sanitized): 'PLK4.pt'
  Processed ID 2/379: original='CDC2L5', sanitized='CDC2L5' -> FOUND (using sanitized): 'CDC2L5.pt'
  Processed ID 3/379: original='AKT2', sanitized='AKT2' -> FOUND (using sanitized): 'AKT2.pt'
  Processed ID 4/379: original='ACVRL1', sanitized='ACVRL1' -> FOUND (using sanitized): 'ACVRL1.pt'
  Processed ID 5/379: original='VRK2', sanitized='VRK2' -> FOUND (using sanitized): 'VRK2.pt'
  Processed ID 6/379: original='PRKX', sanitized='PRKX' -> FOUND (using sanitized): 'PRKX.pt'
  Processed ID 7/379: original='DAPK1', sanitized='DAPK1' -> FOUND (using sanitized): 'DAPK1.pt'
  Processed ID 8/379: original='EPHB6', sanitized='EPHB6' -> FOUND (using sanitized): 'EPHB6.pt'
  Processed ID 9/379: original='EPHA4', sanitized='EPHA4' -> FOUND (using sanitized): 'EPHA4.pt'
  Processed ID 10/379: original='MYO

Processing drugs: 100%|██████████| 68/68 [00:00<00:00, 621.09it/s]


...

Epoch 246/500 | LR: 3.91e-07 | Train L=0.3126 CI=0.7130 MSE=0.57 MAE=0.49 | Val L=0.2119 CI=0.8098 MSE=0.39 MAE=0.34


Evaluating	: 100%|██████████| 21/21 [00:03<00:00,  5.89it/s]


Epoch 247/500 | LR: 3.91e-07 | Train L=0.3124 CI=0.7111 MSE=0.57 MAE=0.50 | Val L=0.2120 CI=0.8098 MSE=0.39 MAE=0.34


Evaluating	: 100%|██████████| 21/21 [00:03<00:00,  5.44it/s]


Epoch 248/500 | LR: 3.91e-07 | Train L=0.3156 CI=0.7041 MSE=0.58 MAE=0.50 | Val L=0.2119 CI=0.8101 MSE=0.39 MAE=0.34

Early stopping triggered after 32 epochs without improvement.


In [6]:
import pandas as pd

from utils.helper_functions import plot_loss_curves

csv_path = f"{training_args.model_path}.csv"
metrics = pd.read_csv(csv_path) 

plot_loss_curves(metrics)

KeyError: 'train_acc'